## Домашнее задание 1

Я взяла корпус Marujo2012 (https://github.com/SDuari/Keyword-Extraction-Datasets/blob/master/Marujo2012.zip). Это датасет новостей на разные темы, размеченный по ключевым словам. На каждый текст в среднем приходится по 69 униграмм и 48 биграмм ключевых слов, все они присутствуют в исходных текстах. Я взяла именно этот датасет, потому что в нём содержатся тексты на разные темы, что должно упростить задачу, и эти тексты не слишком длинные и не слишком короткие. Ключевые слова содержатся списком в отдельных файлах с соответствующими названиями. В корпус я вручную скопировала по одному файлу новости и одному файлу с ключевыми словами для каждой тематической категории (Кроме политики и бизнеса. Надо было уменишь объём корпуса, и я выкинула самые неинтересные мне темы)

In [266]:
import os
import pandas as pd

import RAKE
import nltk
import numpy as np

from pprint import pprint
from summa import keywords
from statistics import mean
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Составить корпус

In [60]:
texts, given_keywords = [], []
count = 0
for filename in os.listdir('dataset'):
    with open(f'dataset/{filename}', encoding='UTF-8') as f:
        data = f.read()
    if filename.endswith('txt'):
        texts.append(data)
        count += len(data.split(' '))
    else:
        given_keywords.append(data.split('\n'))

print(f'Всего текстов: {len(texts)}')
print(f'Всего токенов: {count}')

Всего текстов: 7
Всего токенов: 3805


In [61]:
print(texts[0])

Singer Michael Buble weds TV star Luisana Lopilato
Canadian pop star Michael Buble kisses his bride Argentine TV actress Luisana Lopilato after their civil wedding ceremony in Buenos Aires, Argentina, Thursday March 31, 2011. (AP Photo/Natacha Pisarenko) BUENOS AIRES, Argentina (AP) - Canadian pop star Michael Buble married Argentine TV actress Luisana Lopilato in a civil ceremony on Thursday. The Grammy-winning singer of ""Crazy Love"" and his Argentine sweetheart posed for a mob of fans after tying the knot at a civil registry in downtown Buenos Aires. She wore a lilac chiffon dress with silver high heels and Buble wore a sharp gray suit as he leaned down for a smooch. Then Lopilato, 23, tossed a bouquet of purple orchids into the crowd as some fans threw rice and red rose petals and a young woman shouted ""I love you Michael!"" Buble, 35, won his Grammy last month for traditional pop vocal album, and is one of North America's top-grossing concert entertainers. Lopilato made her name

In [62]:
df = pd.DataFrame({'texts': texts,
                   'given_keywords': given_keywords})

In [63]:
df.head(5)

,texts,given_keywords
0,Singer Michael Buble weds TV star Luisana Lopi...,"[married Argentine, his Grammy, ceremony, kiss..."
1,Dolce and Gabbana will not face trial after &#...,[masterminded an instantly recognisable highen...
2,Plus size fashion: Size 18 clothes CAN look st...,"[earrings 650, ASOScom, 28 Indigo Collection, ..."
3,Radioactive boars in Germany a legacy of Chern...,"[typically conservative, snouts, radioactivity..."
4,Are Some Kids Overscheduled?\nExperts debate p...,"[Mata, American kids always, presentation Thur..."


#### Разметка - сразу в тетрадке

In [64]:
my_keywords = []

In [65]:
for text in df.texts.to_list():
    print('\n', text)
    my_answer = input('\nKeywords: ')
    my_keywords.append(my_answer.split(', '))


 Singer Michael Buble weds TV star Luisana Lopilato
Canadian pop star Michael Buble kisses his bride Argentine TV actress Luisana Lopilato after their civil wedding ceremony in Buenos Aires, Argentina, Thursday March 31, 2011. (AP Photo/Natacha Pisarenko) BUENOS AIRES, Argentina (AP) - Canadian pop star Michael Buble married Argentine TV actress Luisana Lopilato in a civil ceremony on Thursday. The Grammy-winning singer of ""Crazy Love"" and his Argentine sweetheart posed for a mob of fans after tying the knot at a civil registry in downtown Buenos Aires. She wore a lilac chiffon dress with silver high heels and Buble wore a sharp gray suit as he leaned down for a smooch. Then Lopilato, 23, tossed a bouquet of purple orchids into the crowd as some fans threw rice and red rose petals and a young woman shouted ""I love you Michael!"" Buble, 35, won his Grammy last month for traditional pop vocal album, and is one of North America's top-grossing concert entertainers. Lopilato made her na


Keywords: fashion, plus size, clothes, size, plus-size, hips, thighs

 Radioactive boars in Germany a legacy of Chernobyl
Radioactive boars: A quarter century after the Chernobyl nuclear disaster in the Soviet Union carried a cloud of radiation across Europe, these animals are radioactive enough that people are urged not to eat them. BERLIN For a look at just how long radioactivity can hang around, consider Germany's wild boars. A quarter century after the Chernobyl nuclear disaster in the Soviet Union carried a cloud of radiation across Europe, these animals are radioactive enough that people are urged not to eat them. And the mushrooms the pigs dine on aren't fit for consumption either. Germany's experience shows what could await Japan - if the problems at the Fukushima Dai-ichi plant get any worse. Nuclear power around the world The German boars roam in forests nearly 950 miles (1,500 kilometers ) from Chernobyl. Yet, the amount ofradioactive cesium-137 within their tissue often re


Keywords: kids, school, children, school, activities, extracurricular

 NCAA tournament:
Nobody will dispute that they are great coaches, and the latest evidence lies in the teams they guided to this year's Final Four. Nobody will argue that John Calipari and Jim Calhoun are saints, either. The men on the sidelines for the Kentucky-Connecticut Final Four matchup have had their share of trouble - with each other, the programs they run and the NCAA. They are a microcosm of everything that's right and wrong in college basketball - a coach-driven game where good leaders can elevate programs and players to new levels but the road to success often produces its fair share of cringeworthy dealing. Calipari leads the fourth-seeded Wildcats (29-8) against Calhoun and the third-seeded Huskies (30-9) in the second semifinal Saturday. Both coaches coaxed a turnaround out of their young, struggling teams to make unexpected trips to the game's biggest stage - the third for Calipari and fourth for Ca

In [66]:
df['my_keywords'] = my_keywords

In [67]:
df.head(5)

,texts,given_keywords,my_keywords
0,Singer Michael Buble weds TV star Luisana Lopi...,"[married Argentine, his Grammy, ceremony, kiss...","[Michael Buble, Luisana Lopilato, pop star, TV..."
1,Dolce and Gabbana will not face trial after &#...,[masterminded an instantly recognisable highen...,"[Dolce, Gabbana, fashion, tax dodge, fraud, tr..."
2,Plus size fashion: Size 18 clothes CAN look st...,"[earrings 650, ASOScom, 28 Indigo Collection, ...","[fashion, plus size, clothes, size, plus-size,..."
3,Radioactive boars in Germany a legacy of Chern...,"[typically conservative, snouts, radioactivity...","[Chernobyl, cesium, meat, becquerels, wild boa..."
4,Are Some Kids Overscheduled?\nExperts debate p...,"[Mata, American kids always, presentation Thur...","[kids, school, children, school, activities, e..."


In [68]:
df.to_csv('df_dataset.csv', index=False)

#### Оценить пересечение, составить эталон

In [124]:
def depunkt(text):
    words = [word.lower() for word in word_tokenize(text)
             if word.isalpha()]
    return ' '.join(words)

In [126]:
df['texts'] = df['texts'].apply(depunkt)
df['given_keywords'] = df['given_keywords'].map(lambda x: list(map(str.lower, x)))
df['my_keywords'] = df['my_keywords'].map(lambda x: list(map(str.lower, x)))

In [108]:
given = df.given_keywords.to_list()
my = df.my_keywords.to_list()

In [109]:
for g, m in zip(given, my):
    print(f'''Готовая разметка {len(g)} шт, моя разметка {len(m)} шт, совпали {len(set(g) & set(m))} шт''')

Готовая разметка 42 шт, моя разметка 10 шт, совпали 1 шт
Готовая разметка 62 шт, моя разметка 8 шт, совпали 4 шт
Готовая разметка 118 шт, моя разметка 7 шт, совпали 3 шт
Готовая разметка 75 шт, моя разметка 12 шт, совпали 9 шт
Готовая разметка 96 шт, моя разметка 6 шт, совпали 2 шт
Готовая разметка 51 шт, моя разметка 5 шт, совпали 1 шт
Готовая разметка 39 шт, моя разметка 6 шт, совпали 4 шт


Я начну с эталона как с объеднинения двух разметок, затем, если результаты будут слишком хорошие, попробую сократить эталон только до моей разметки. Пока мне не нравится, что в готовой разметке уж очень много ключевых слов, которые, на мой взгляд, совсем и не ключевые

In [110]:
gold = []
for g, m in zip(given, my):
    union = list(set(g) | set(m))
    gold.append(union)

In [130]:
df['gold'] = gold

#### Применить три метода извлечения ключевых слов

In [131]:
# rake

rake_keywords = []
rake = RAKE.Rake(stop)
for text in df.texts:
    rake_output = rake.run(text)#, maxWords=3, minFrequency=2)
    wordlist = list(map(lambda x: x[0], filter(lambda x: x[1] != 0, rake_output)))
    rake_keywords.append(wordlist)
    
df['rake_keywords'] = rake_keywords

In [132]:
# textrank

textrank_keywords = []
for text in df.texts:
    textrank_output = keywords.keywords(text, language='english', additional_stopwords=stop, scores=True)
    wordlist = list(map(lambda x: x[0], filter(lambda x: x[1] != 0, textrank_output)))
    textrank_keywords.append(wordlist)
    
df['textrank_keywords'] = textrank_keywords

In [264]:
# tf-idf

tfidf_keywords = []

vectorizer = TfidfVectorizer(ngram_range=(1,3), stop_words=stop)
X = vectorizer.fit_transform(df.texts).toarray()
all_words = np.array(vectorizer.get_feature_names())
for idx, row in enumerate(X):
    top_words = np.argsort(row)[::-1]
    # беру столько же топ слов, сколько в золотом стандарте
    sorted_words = all_words[top_words[:len(df.gold[idx])].ravel()]
    tfidf_keywords.append(sorted_words)
    
df['tfidf_keywords'] = tfidf_keywords

#### Составить и применить шаблоны
Я одновременно размечаю ключевые слова по частям речи автоматически и просматриваю результаты глазами. На самом деле делаю это потому, что не сильно разбираюсь в частях речи в английском с их почти отсутствующей морфологией

In [189]:
def get_pos(phrase_list):
    res = []
    for phrase in phrase_list:
        pos_list = list(map(lambda x: x[1], nltk.pos_tag(phrase.split(' '))))
        res.append(' + '.join(pos_list))
    return res

In [196]:
# снова вручную сразу в тетрадке
patterns = []
for kw_list in gold:
    phrases = [kw for kw in kw_list if len(kw.split(' ')) > 1]
    pprint(list(zip(phrases, get_pos(phrases))))
    pattern = input('\nPattern: ')
    patterns.extend(pattern.split(', '))

[('south american', 'JJ + JJ'),
 ('after tying', 'IN + VBG'),
 ('luisana lopilato', 'NN + NN'),
 ('bride argentine tv actress', 'RB + JJ + NN + NN'),
 ('argentina ap', 'NNS + VBP'),
 ('pop star', 'NN + NN'),
 ('wedding ceremony', 'NN + NN'),
 ('canadian pop', 'JJ + NN'),
 ('tv actress', 'NN + NN'),
 ('buenos aires', 'NN + NNS'),
 ('married argentine', 'JJ + NN'),
 ('buble dated', 'JJ + VBD'),
 ('his grammy', 'PRP$ + NN'),
 ('lopilato made', 'NN + VBD'),
 ('vocal album', 'JJ + NN'),
 ('argentine sweetheart', 'JJ + NN'),
 ('civil registry', 'JJ + NN'),
 ('michael buble', 'NN + NN'),
 ('crazy love', 'JJ + NN'),
 ('chiffon dress', 'NN + NN')]

Pattern: JJ + JJ, NN + NN, JJ + NN
[('milanese fashion notion', 'JJ + NN + NN'),
 ('giorgio armani accused', 'NN + NNS + VBD'),
 ('tax dodge', 'NN + NN'),
 ('exalting gang rape', 'VBG + NN + NN'),
 ('supersexy ads', 'NN + NNS'),
 ('younger line dg', 'JJR + NN + NN'),
 ('welltailored suiting', 'VBD + NN'),
 ('around 1bn', 'IN + CD'),
 ('preliminary he

In [198]:
# убрать дубликаты
patterns = list(set(patterns))

In [230]:
# добавить части речи для одиночных слов
single_pos = ['NN', 'NNS', 'JJ', 'JJS', 'NNP', 'NNPS', 'VBN']
patterns.extend(single_pos)

In [232]:
patterns

['NN + CD',
 'JJ + NN + IN + NN + NN',
 'JJ + NNS',
 'NN + CC + NN',
 'NN',
 'NN + NNS',
 'VBD + NN',
 'VBN + NN',
 'JJS',
 'JJ + NN + NN + NN',
 'CC + NN',
 'RB + JJ + NNS',
 'JJ + NN',
 'DT + JJ + NNS',
 'VBN',
 'NN + NN + NN',
 'NNP',
 'NNS + NNS',
 'NN + NN',
 'JJ + JJ',
 'NNS + NN',
 'CD + NN',
 'NNS + JJ',
 'NNPS',
 'NNS',
 'JJ']

In [233]:
def filter_kw(kw_list):
    new_kw_list = []
    for kw in kw_list:
        pos_list = list(map(lambda x: x[1], nltk.pos_tag(kw.split(' '))))
        comb = ' + '.join(pos_list)
        if comb in patterns:
            new_kw_list.append(kw)
    return new_kw_list

In [278]:
df['gold_filtered'] = df['gold'].apply(filter_kw)
df['my_keywords_filtered'] = df['my_keywords'].apply(filter_kw)
df['rake_keywords_filtered'] = df['rake_keywords'].apply(filter_kw)
df['textrank_keywords_filtered'] = df['textrank_keywords'].apply(filter_kw)
df['tfidf_keywords_filtered'] = df['tfidf_keywords'].apply(filter_kw)

In [281]:
df.head(2)

,texts,given_keywords,my_keywords,gold,rake_keywords,textrank_keywords,tfidf_keywords,rake_keywords_filtered,textrank_keywords_filtered,tfidf_keywords_filtered,gold_filtered,my_keywords_filtered
0,singer michael buble weds tv star luisana lopi...,"[married argentine, his grammy, ceremony, kiss...","[michael buble, luisana lopilato, pop star, tv...","[ceremony, mansion, south american, after tyin...","[south american concert tour lopilato, twitter...","[lopilato, argentine, buble weds tv star, conc...","[lopilato, buble, buenos, buenos aires, aires,...","[lilac chiffon dress, silver high heels, met l...","[lopilato, argentine, concert, buenos, pop, ac...","[lopilato, buble, buenos, buenos aires, aires,...","[ceremony, mansion, south american, luisana lo...","[michael buble, luisana lopilato, pop star, tv..."
1,dolce and gabbana will not face trial after fr...,[masterminded an instantly recognisable highen...,"[dolce, gabbana, fashion, tax dodge, fraud, tr...","[2005, couple, milanese fashion notion, giorgi...","[charges judge simone luerti ruled, world larg...","[dolce, design, brands, brand, branded, italia...","[dolce, gabbana, dolce gabbana, accused, judge...","[milanese designers, scandal dolce, charge dol...","[dolce, design, brands, brand, branded, italia...","[dolce, gabbana, dolce gabbana, accused, judge...","[couple, tax dodge, fraud, supersexy ads, dolc...","[dolce, gabbana, fashion, tax dodge, fraud, tr..."


#### Оценить метрики

In [274]:
def metrics(gold_col, predicted_col):
    precisions, recalls, f_scores = [], [], []
    for gold, predicted in zip(gold_col, predicted_col):
        precision = len(set(gold) & set(predicted)) / len(predicted)
        recall = len(set(gold) & set(predicted)) / len(gold)
        try:
            f_score = 2 * (precision * recall) / (precision + recall)
        except ZeroDivisionError:
            f_score = 0
        precisions.append(precision)
        recalls.append(recall)
        f_scores.append(f_score)
        
    return [mean(precisions), mean(recalls), mean(f_scores)]

In [291]:
metrics_df = pd.DataFrame({'metric': ['precision', 'recall', 'f-score'],
                           'rake_nf': metrics(df.gold, df.rake_keywords),
                           'rake_f': metrics(df.gold_filtered, df.rake_keywords_filtered),
                           'textrank_nf': metrics(df.gold, df.textrank_keywords),
                           'textrank_f': metrics(df.gold_filtered, df.textrank_keywords_filtered),
                           'tfidf_nf': metrics(df.gold, df.tfidf_keywords),
                           'tfidf_f': metrics(df.gold_filtered, df.tfidf_keywords_filtered)})
metrics_df.set_index('metric')
metrics_df.style.highlight_max(color = 'lightgreen', axis = 1)

,metric,rake_nf,rake_f,textrank_nf,textrank_f,tfidf_nf,tfidf_f
0,precision,0.234613,0.362106,0.332706,0.357269,0.247200,0.349172
1,recall,0.382878,0.441043,0.115772,0.133007,0.247200,0.304688
2,f-score,0.281640,0.382527,0.169096,0.191688,0.247200,0.324029


#### Описать ошибки

Лучше всего по всем метрикам сработал метод rake + фильтры - 0.36 точность, 0.44 полнота, 0.38 f-score.  
Дальше по точности идёт textrank с фильтрами и tfidf с фильтрами.  
По полноте rake без фильтров и tf-idf с фильтрами.  
По ф-мере tf-idf с фильтрами и rake без фильтров.

Добавление фильтров для каждого из методов улучшило их результаты по всем метрикам

Посмотрим внимательнее на все слова:

In [292]:
pd.set_option('display.max_colwidth', None)

In [293]:
df

texts  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               singer michael buble weds tv star luisana lopilato canadian pop star michael buble kisses his bride argentine tv actress luisana lopilato after their civil wedding ceremony in buenos aires argentina thursday march ap pisarenko buenos aires argentina ap canadian pop star michael buble married argentine tv actress luisana lopilato in a civil c

Есть ощущение, что все три метода выделяют ключевые слова лучше, чем было в изначальной разметке, данной с датасетом. Особенно хорошо это заметно, если смотреть на отфильтрованные варианты методов. В золотом стандарте, состоящем в основном из ключевых слов, размеченных не мной, встречается много странных словосочетаний, например, одиночных наречий или глаголов, или сочетаний с притяжательными местоимениями или теми же глаголами. В ключевых словах мы хотим видеть скорее существительные, одиночные или с каким-то зависимыми. Получается, несмотря на относительно низкие значения метрик, проблема не в выдаче алгоритмов, которая мне кажется разумной, а в референсных ключевых словах, заданных не мной.  

Можно попробовать сравнить только со словами, которые задала я (но надо помнить, что их очень мало, потому что мне не хватило фантазии. было бы больше осмысленно выделенных слов - и результат метрик был бы наверянка выше):

In [295]:
another_metrics_df = pd.DataFrame({'metric': ['precision', 'recall', 'f-score'],
                           'rake_nf': metrics(df.my_keywords, df.rake_keywords),
                           'rake_f': metrics(df.my_keywords_filtered, df.rake_keywords_filtered),
                           'textrank_nf': metrics(df.my_keywords, df.textrank_keywords),
                           'textrank_f': metrics(df.my_keywords_filtered, df.textrank_keywords_filtered),
                           'tfidf_nf': metrics(df.my_keywords, df.tfidf_keywords),
                           'tfidf_f': metrics(df.my_keywords_filtered, df.tfidf_keywords_filtered)})
another_metrics_df.set_index('metric')
another_metrics_df.style.highlight_max(color = 'lightgreen', axis = 1)

,metric,rake_nf,rake_f,textrank_nf,textrank_f,tfidf_nf,tfidf_f
0,precision,0.041300,0.072081,0.113853,0.131355,0.086878,0.127071
1,recall,0.542177,0.550835,0.383673,0.379344,0.726871,0.737693
2,f-score,0.074560,0.121411,0.171837,0.191543,0.152822,0.212977


С таким золотым стандартом гораздо выше стала полнота, у tf-idf она достигла максимальных 0.74 балла. Общая ф-мера тоже максимальная для tf-idf, а точность - для textrank с фильтрами.  
Но в целом точность ощутимо упала для всех методов. Я связываю это с тем, что все алгоритмы предсказывали много ключевых слов, намного больше, чем в этом золотом стандарте. Логично, что чем больше слов называть, чем больше раз пытаться угадать заданные слова, тем больше будет и ложнопозитивных, и ложноотрицательных ответов, соответственно, точность будет падать, а полнота - расти. Однако для распознавания слов, на мой взгляд, точность важнее, потому что мы хотим снизить число ложнопозитивных ответов: нам лучше упустить какие-то релевантные ключевые слова, чем запутать пользователя, назвав ему слова, не передающие смысл текста.    

В целом получается, что проблема в этой задаче - не алгоритмы, а референсные стандарты. Единственное дополнение, которое мне видится возможным для текстов, - добавить лемматизацию к преобработке, что в английских существительных будет выражаться в основном в приведении к единственному числу. Я не стала делать этого сразу, чтобы сделать ключевые слова читаемыми и понятными для человека, чтобы, например, clothes и jeans не превратились в cloth и jean. Однако это может помочь алгоритмам лучше учитывать слова, которые встречаются в текстах и в единственном, и во множенственном числе.